## Toronto Neighborhood Clustering

This notebook explores and clusters the neighborhoods in Toronto.
There are 3 parts to this notebook:
1. scraping and cleaning the data from wikipedia table with beautiful soup
2. adding the latitude, longitude data using pgeocode 
3. calling Foursquare API to find neighborhood's characteristics (by most common venue) and use k-means to cluster the neighborhoods

In [ ]:
# before we begin, import libraries

# library to handle data in a vectorized manner
import numpy as np 

 # library for data analsysis
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle JSON files
import json 

# library for convert an address into latitude and longitude values
# !pip install geopy 
# from geopy.geocoders import Nominatim 

# library for convert a postal code into geographical details including latitude and longitude values
# geocoder/ csv file didn't work
!pip install pgeocode
import pgeocode


# library to handle requests
import requests 
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5.0 
import folium # map rendering library

# used for scraping
import urllib.request 
from bs4 import BeautifulSoup


print('Libraries imported.')



### Part 1: scraping & cleaning data 

In [280]:
# 1.1 start with scraping wikipedia table
# instructions here https://alanhylands.com/how-to-web-scrape-wikipedia-python-urllib-beautiful-soup-pandas/

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

# the following codes are for locating the right data table
# uncheck to check the syntax

# print(soup.prettify())
# all_tables=soup.find_all("table")
# all_tables


In [281]:
# 1.2 using Pandas read html to turn wikitable into df
data_table=soup.find('table', class_='wikitable sortable')
data_table

pdf = pd.read_html(str(data_table))[0]
pdf.head()


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [282]:
# 1.3 cleaning up pdf 
# 1.3.1. Ignore cells with a borough that is Not assigned
toronto = pdf[pdf['Borough']!= 'Not assigned'].reset_index(drop=True)

# 1.3.2. rename column without space
toronto.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

toronto.head()


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [283]:
# 1.3.3. check for duplicated postal code
print('Duplicated postal code check:', toronto.duplicated(subset=['PostalCode']).any())

# 1.3.4. check if Borough column has NA values
check1 = toronto[toronto['Borough'] == 'Not assigned'].reset_index(drop=True)
print('\nBorough NA check:',check1.head())

# 1.3.5. check if Neighbourhood column has NA values
check2 = toronto[toronto['Neighbourhood'] == 'Not assigned'].reset_index(drop=True)
print('\nNeighbourhood NA check:',check2.head())


Duplicated postal code check: False

Borough NA check: Empty DataFrame
Columns: [PostalCode, Borough, Neighbourhood]
Index: []

Neighbourhood NA check: Empty DataFrame
Columns: [PostalCode, Borough, Neighbourhood]
Index: []


In [290]:
# check data's final shape
print(toronto.shape)
toronto.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


This project was made by Jane Goh 30 Oct 2020 as part of the Coursera IBM Data Science Professional Capstone Project

### Part 2: Adding neighbourhood latitude, longitude

In [285]:
# 2.1. create empty df
gdf = pd.DataFrame()

In [286]:
# 2.2. use for loop to find all the neighborhoods' geographical details using pgeocode's find by postal code function
for i in range(toronto.shape[0]):
    nomi = pgeocode.Nominatim('CA')
    gdf = gdf.append(nomi.query_postal_code(str(toronto['PostalCode'][i])))
    
gdf.head()


,accuracy,community_code,community_name,country_code,county_code,county_name,latitude,longitude,place_name,postal_code,state_code,state_name
0,1.0,NaN,NaN,CA,NaN,North York,43.7545,-79.3300,North York (York Heights / Victoria Village / ...,M3A,ON,Ontario
0,6.0,NaN,NaN,CA,NaN,NaN,43.7276,-79.3148,North York (Sweeney Park / Wigmore Park),M4A,ON,Ontario
0,6.0,NaN,NaN,CA,8133394.0,Toronto,43.6555,-79.3626,Downtown Toronto (Regent Park / Port of Toronto),M5A,ON,Ontario
0,6.0,NaN,NaN,CA,NaN,North York,43.7223,-79.4504,North York (Lawrence Manor / Lawrence Heights),M6A,ON,Ontario
0,NaN,NaN,NaN,CA,NaN,NaN,43.6641,-79.3889,Queen's Park Ontario Provincial Government,M7A,ON,Ontario


In [ ]:
# 2.3. take only the postal code, latitude, longitude columns
gdfll = gdf[['postal_code','latitude','longitude']]
gdfll.rename(columns={'postal_code':'PostalCode','latitude':'Latitude','longitude':'Longitude'},inplace=True)

# 2.4. join the tables together so the toronto neighborhood data has latitude, longitude data
toronto_gdf = toronto
toronto_gdf = toronto_gdf.join(gdfll.set_index('PostalCode'), on='PostalCode')
toronto_gdf.drop_duplicates(inplace=True)

In [288]:
# check final data 
toronto_gdf.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


This project was made by Jane Goh 30 Oct 2020 as part of the Coursera IBM Data Science Professional Capstone Project

### Part 3: Explore & cluster Toronto neighbourhoods 

In [236]:
# Prep1. The original data got 103 neighborhoods, we'll work only with boroughs that contain the word 'Toronto'
Toronto_data = toronto_gdf[toronto_gdf['Borough'].str.contains('Toronto')].reset_index(drop=True)
print(Toronto_data.shape)
Toronto_data.head()


(39, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


In [237]:
# Prep2. take a look at the coverage on map
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location= [43.6487, -79.38544], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 3.1 Foursquare data

In [277]:
# We are working with Foursquare API to get the neighborhoods details
# below are the parameters to be passed
CLIENT_ID ='YourClientIDHere'
CLIENT_SECRET ='YourClientSecretHere'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YourClientIDHere
CLIENT_SECRET:YourClientSecretHere


In [239]:
# check to see the data we are working with is correct
Toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [240]:
# 3.1.1. define new function to get nearby venues from Foursquare API based on 
# neighborhood name, latitude, longitude and within radius of 500m 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [241]:
# 3.1.2. get the Toronto neighborhood venues from Fourquare 

toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [243]:
# 3.1.3. check the results df
print(toronto_venues.shape)
toronto_venues.head()

(1547, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa


In [244]:
# 3.1.4. Tabulate the venues results
print(toronto_venues.groupby('Neighborhood').count().shape)
toronto_venues.groupby('Neighborhood').count()

(39, 6)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,90,90,90,90,90,90
"Brockton, Parkdale Village, Exhibition Place",37,37,37,37,37,37
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",57,57,57,57,57,57
Central Bay Street,74,74,74,74,74,74
Christie,12,12,12,12,12,12
Church and Wellesley,74,74,74,74,74,74
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,22,22,22,22,22,22


In [245]:
# It seems there are neighborhoods with few results, that's not meaningful when we do clustering
# let's remove the neighborhoods with insufficient number of results, say less than 30
# 3.1.5. find neighborhoods with < 30 venues

L30 = pd.DataFrame(toronto_venues.groupby('Neighborhood').count())
dropV = L30[L30['Venue'] <= 30].index
dropV


Index(['Business reply mail Processing Centre, South Central Letter Processing Plant Toronto',
       'Christie', 'Davisville', 'Davisville North',
       'Dufferin, Dovercourt Village',
       'Forest Hill North & West, Forest Hill Road Park',
       'Harbourfront East, Union Station, Toronto Islands',
       'High Park, The Junction South', 'India Bazaar, The Beaches West',
       'Lawrence Park', 'Moore Park, Summerhill East',
       'North Toronto West, Lawrence Park',
       'Queen's Park, Ontario Provincial Government',
       'Regent Park, Harbourfront', 'Rosedale', 'Roselawn', 'Studio District',
       'Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park',
       'The Annex, North Midtown, Yorkville', 'The Beaches',
       'University of Toronto, Harbord'],
      dtype='object', name='Neighborhood')

In [246]:
# 3.1.6. drop the neighborhoods with < 30 venues from the toronto_venues df

toronto_venues = toronto_venues[~toronto_venues.Neighborhood.isin(dropV)]
toronto_venues.shape

(1306, 7)

In [247]:
# 3.1.7. check the toronto_venues df again, the shape has decreased

print(toronto_venues.groupby('Neighborhood').count().shape)
toronto_venues.groupby('Neighborhood').count()

(18, 6)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,90,90,90,90,90,90
"Brockton, Parkdale Village, Exhibition Place",37,37,37,37,37,37
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",57,57,57,57,57,57
Central Bay Street,74,74,74,74,74,74
Church and Wellesley,74,74,74,74,74,74
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Kensington Market, Chinatown, Grange Park",64,64,64,64,64,64


In [248]:
# 3.1.8. Now we will do one-hot encoding of the Toronto venues categories for each venue

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Garden District, Ryerson",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Garden District, Ryerson",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Garden District, Ryerson",0,0,0,0,0,0,0,0,0,0,0

In [249]:
# double check the shape is the same as toronto_venues, no data missing
toronto_onehot.shape

(1306, 198)

In [250]:
# 3.1.9. group the neighborhoods together and find each neighborhood's average # of venues per category 
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.011111,0.000000,0.000000,0.011111,0.022222,0.000000,0.000000,0.000000,0.000000,0.011111,0.044444,0.000000,0.000000,0.011111,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.022222,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.044444,0.000000,0.000000,0.022222,0.000000,0.000000,0.000000,0.022222,0.100000,0.000000,0.00,0.00,0.011111,0.000000,0.011111,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.022222,0.011111,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.011111,0.000000,0.000000,0.000000,0.000000,0.022222,0.011111,0.000000,0.011111,0.000000,0.000000,0.00,0.011111,0.011111,0.011111,0.000000,0.000000,0.000000,0.00000,0.00,0.011111,0.000000,0.000000,0.00,0.000000,0.00,0.011111,0.011111,0.011111,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.055556,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.033333,0.011111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.011111,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,0.00,0.000000,0.000000,0.011111,0.00

In [251]:
# checking the number of neighborhoods and venues categories are same
toronto_grouped.shape

(18, 198)

In [252]:
# 3.1.10. define new function to return the most common venue categories in each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [253]:
# 3.1.11. let's find each neighborhood's top 10 venues

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Bakery,Café,Japanese Restaurant,Seafood Restaurant,Restaurant,Beer Bar,Sandwich Place,Deli / Bodega
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Thrift / Vintage Store,Breakfast Spot,Gift Shop,Hawaiian Restaurant,Boutique,Brewery,French Restaurant,Liquor Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Bar,Café,Park,Speakeasy,French Restaurant,Gym / Fitness Center,Bank,Bakery
3,Central Bay Street,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Restaurant,Bubble Tea Shop,Sandwich Place,Breakfast Spot,Hotel,Middle Eastern Restaurant
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Café,Bubble Tea Shop,Men's Store,Mediterranean Restaurant,Yoga Studio


#### 3.2 Clustering

In [254]:
# Now we are ready to do the clustering and analysis of neighborhoods
# 3.2.1. run the clustering algorithm

# set number of clusters
kclusters = 5

# dropping the 'Neighborhood' column cause we don't need it for the clustering algorithm
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 1, 1, 0, 0, 0, 0, 0, 4, 4])

In [ ]:
# 3.2.2. Making a new df toronto_merged from the original Toronto_data BUT!
# Remember we removed the neighborhoods with < 30 venue results in (3.1.5)? 
# We also gotta do that to the Toronto_data - drop neighborhoods < 30 venue results 

toronto_merged = Toronto_data[~Toronto_data.Neighbourhood.isin(dropV)]
toronto_merged.shape

In [259]:
# 3.2.3. merge the Toronto neighbourhood data with cluter label from (3.2.1) and most common venues data from (3.1.11)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Hotel,Fast Food Restaurant,Diner,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,0,Coffee Shop,Café,Seafood Restaurant,Restaurant,Cocktail Bar,American Restaurant,Beer Bar,Gastropub,Bakery,Moroccan Restaurant
5,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,0,Coffee Shop,Hotel,Bakery,Café,Japanese Restaurant,Seafood Restaurant,Restaurant,Beer Bar,Sandwich Place,Deli / Bodega
6,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860,0,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Restaurant,Bubble Tea Shop,Sandwich Place,Breakfast Spot,Hotel,Middle Eastern Restaurant
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6496,-79.3833,0,Coffee Shop,Café,Gym,Hotel,Asian Restaurant,Steakhouse,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Restaurant
11,M6J,West Toronto,"Little Portugal, Trinity",43.6480,-79.4177,4,Coffee Shop,Bar,Restaurant,Vietnamese Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Asian Restaurant,Yoga Studio,Diner,Record Shop
12,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Café,Ice Cream Shop,Lounge,Liquor Store,Pub,Bubble Tea Shop
13,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.6469,-79.3823,0,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Japanese Restaurant,Beer Bar,Sporting Goods Shop
14,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.6383,-79.4301,1,Coffee Shop,Café,Thrift / Vintage Store,Breakfast Spot,Gift Shop,Hawaiian Restaurant,Boutique,Brewery,French Restaurant,Liquor Store
16,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.6492,-79.3823,0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Japanese Restaurant,Asian Restaurant,Deli / Bodega,Salad Place


In [270]:
# 3.2.4 visualize the clusters with map

# create map
map_clusters = folium.Map(location= [43.6629, -79.3957], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 3.3 Examine Clusters

This is the results of Toronto Neighbourhood Clustering Project, appreciate your constructive feedback!

In [271]:
# cluster 1: White Collar District/ Downtown/ Tourist Area
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Garden District, Ryerson",0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Hotel,Fast Food Restaurant,Diner,Pizza Place
3,St. James Town,0,Coffee Shop,Café,Seafood Restaurant,Restaurant,Cocktail Bar,American Restaurant,Beer Bar,Gastropub,Bakery,Moroccan Restaurant
5,Berczy Park,0,Coffee Shop,Hotel,Bakery,Café,Japanese Restaurant,Seafood Restaurant,Restaurant,Beer Bar,Sandwich Place,Deli / Bodega
6,Central Bay Street,0,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Restaurant,Bubble Tea Shop,Sandwich Place,Breakfast Spot,Hotel,Middle Eastern Restaurant
8,"Richmond, Adelaide, King",0,Coffee Shop,Café,Gym,Hotel,Asian Restaurant,Steakhouse,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Restaurant
13,"Toronto Dominion Centre, Design Exchange",0,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Japanese Restaurant,Beer Bar,Sporting Goods Shop
16,"Commerce Court, Victoria Hotel",0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Japanese Restaurant,Asian Restaurant,Deli / Bodega,Salad Place
34,Stn A PO Boxes,0,Coffee Shop,Restaurant,Gym,Hotel,Italian Restaurant,Sporting Goods Shop,Deli / Bodega,Café,Japanese Restaurant,Breakfast Spot
36,"First Canadian Place, Underground city",0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Japanese Restaurant,Asian Restaurant,Deli / Bodega,Salad Place
37,Church and Wellesley,0,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Café,Bubble Tea Shop,Men's Store,Mediterranean Restaurant,Yoga Studio


In [272]:
# cluster 2: The Old Town/ Hang-Out Town/ Residential Neighborhood
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Brockton, Parkdale Village, Exhibition Place",1,Coffee Shop,Café,Thrift / Vintage Store,Breakfast Spot,Gift Shop,Hawaiian Restaurant,Boutique,Brewery,French Restaurant,Liquor Store
28,"Runnymede, Swansea",1,Café,Coffee Shop,Pizza Place,Bank,Restaurant,Pub,Italian Restaurant,Sushi Restaurant,Falafel Restaurant,Dessert Shop
32,"CN Tower, King and Spadina, Railway Lands, Har...",1,Italian Restaurant,Coffee Shop,Bar,Café,Park,Speakeasy,French Restaurant,Gym / Fitness Center,Bank,Bakery
35,"St. James Town, Cabbagetown",1,Coffee Shop,Restaurant,Café,Park,Pizza Place,Bakery,Italian Restaurant,Market,Chinese Restaurant,Library


In [273]:
# cluster 3: The Convenient Residential District
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"The Danforth West, Riverdale",2,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Café,Ice Cream Shop,Lounge,Liquor Store,Pub,Bubble Tea Shop


In [274]:
# cluster 4: The Young Family Neighborhood
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,"Parkdale, Roncesvalles",3,Coffee Shop,Sushi Restaurant,Eastern European Restaurant,Bakery,Bookstore,American Restaurant,Grocery Store,Gift Shop,Café,Pizza Place


In [275]:
# cluster 5: International District
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"Little Portugal, Trinity",4,Coffee Shop,Bar,Restaurant,Vietnamese Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Asian Restaurant,Yoga Studio,Diner,Record Shop
30,"Kensington Market, Chinatown, Grange Park",4,Mexican Restaurant,Café,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Pizza Place,Burger Joint,Bakery,Thai Restaurant,Dumpling Restaurant


This project was made by Jane Goh 30 Oct 2020 as part of the Coursera IBM Data Science Professional Capstone Project